# Profiling and performance

One of the main reasons people don't use ABMs is because they can be very slow. While "vanilla Starsim" is quite fast (10,000 agents running for 100 timesteps should take about a second), custom modules, if not properly written, can be quite slow.

The first step of fixing a slow module is to identify the problem. To do this, Starsim includes some built-in profiling tools.

Let's look at a simple simulation:

In [ ]:
import sciris as sc
import starsim as ss
sc.options(jupyter=True)

pars = dict(
    start = '2000-01-01',
    stop = '2020-01-01',
    diseases = 'sis',
    networks = 'random'
)

# Profile sim
sim = ss.Sim(pars)
prof = sim.profile()

This graph (which is a shortcut to `sim.loop.plot_cpu()`) shows us how much time each step in the integration loop takes. We can get line-by-line detail of where each function is taking time, though:

In [ ]:
prof.disp(maxentries=5)

(Note that the names of the functions here refer to the *actual* functions called, which may not match the graph above. That's because, for example, `ss.SIS` does not define its own `step()` method, but instead inherits `step()` from `Infection`. In the graph, this is shown as `sis.step()`, but is listed in the table as `Infection.step()`. This is because it's referring to the actual code being run, so refers to where those lines of code exist in the codebase; there is no code corresponding to `SIS.step()` since it's just inherited from `Infection.step()`.)

If you want more detail, you can also define custom functions to follow. For example, we can see that `ss.SIS.infect()` takes the most time in `ss.SIS.step()`, so let's profile that:

In [ ]:
prof = sim.profile(follow=ss.SIS.infect, plot=False)
prof.disp()

(Note: you can only follow functions that are called as part of `sim.run()` this way. To follow other functions, such as those run by `sim.init()`, you can use `sc.profile()` directly.)